In [38]:
%pip install optax
%pip install flax
%pip install --upgrade "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html


[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.5/598.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 39.8 MB/s eta 0:00:00MB/s eta 0:00:01

[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [39]:
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap, jacfwd
import optax
import numpy as np
import matplotlib.pyplot as plt
import flax

In [40]:
# verify JAX is running on GPU and not CPU
print(f"Using the: {jax.devices()[0]}")

Using the: TFRT_CPU_0


In [41]:
# Material Properties 
C1 = 1.0 # Shear modulus
D1 = 10.0 # Bulk modulus

L, H = 1.0, 0.5 # lenght and height

In [42]:
# Training parameters
LEARNING_RATE = 1e-3
Iterations = 5000
N_Domain = 1000 # number of random points inside the square
N_Boundary = 1000 # number of random points on the boundary

In [58]:
# Introducing NN
from flax import linen as nn # NN library for JAX 

class Model_DEM(nn.Module):
    @nn.compact
    def __call__(self, x):
        # Normalizing input
        x_norm = x / jnp.array([L, H])
        activation = (x_norm - 0.5) * 2.0

        # Create Layers
        activation = nn.Dense(128)(activation)
        activation = nn.tanh(activation)
        activation = nn.Dense(128)(activation)
        activation = nn.tanh(activation)
        activation = nn.Dense(128)(activation)
        activation = nn.tanh(activation)

        # Network Output
        output = nn.Dense(2)(activation) # 2 outputs u and v
        return output 

In [59]:
# Initalize NN
model = Model_DEM()

In [60]:
# Create ky for weight init
key = jax.random.PRNGKey(42)

In [70]:
# Define input size of the NN
# input_first = [1.0, 1.0]
input_first = jnp.ones((2,)) # 2 inputs of x and y

In [71]:
params = model.init(key, input_first) # flex automatically creates a first layer of 2x128

In [72]:
# forward of the network
def forward(params, x):
    return model.apply(params, x)

Now I am introducing two parts way to calculate energy

energy_density_point: will calculate energy equation at a specific point (x, y) 

get_energy_point_batch:uses a vmap to run that calculation across our domain of interest

basically it should create a tensor of 1000x2x2

In [82]:
# energy 
def energy_denstiy_point(params, coordinates):
    # This function calculates strain energy (W) at a specfici point (x, y)

    
    # Calculate displacement gradient u
    #jackfwd computes 2x2 jacobian matrix [du/dx, du/dy] [dv/dx, dv/dy]
    # coordiantes is the x_cord and y_cord
    grad_u = jackfwd(forward, coordinates)(params, x_point)

    # Calculate deformation gradient F
    # F = I + div(u)
    I = jnp.eye(2)
    F = I + grad_u

    # Calculate right Cauchy-Green Deformation tensor
    # C = F^T * F
    C_tensor = jnp.dot(F.T, F) # F.T = F^T as a build in JAX feature

    # Calculate Invariants
    # I1 = Trace of C
    I1 = jnp.trace(C_tensor)

    # J = det(F)
    J = jnp.linalg.det(F) # linalg.det is a built in JAX function that computes determinate of an array

    # Determine and outline Neo Hookean Energy Equation
    W = C1 * (I1 - 2.0 - 2.0 * jnp.log(J)) + D1 * pow((J-1), 2)

    return W


In [87]:
#vmap loops function over a list of points
# None --> means same parameters for every point
# 0 --> split coordinates along each coordiantes
get_energy_point_batch = vmap(energy_denstiy_point, in_axes = (None, 0))

In [89]:
# Repeating the same logic but now for the displacement intsead of W
get_displacement_batch = vmap(forward, in_axes = (None, 0))